In [4]:
# HYPOTHESYS
# We expect that mentions of Trumps will have 50/50 positive and negative
# We expect more positive tweets volume from republican states
# We expect Trump's tweets to be majority negative
# What are the main keywords Trump uses and what are the sentiment about those subjects

In [26]:
# import dependencies 
import os
import pandas as pd
from matplotlib import pyplot as plt
import time
import csv
import json
import requests
from config import api_key, api_secret_key
import gmaps
import re 
import twitter
import tweepy 
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
from textblob import TextBlob 
import base64
import string


# Access maps with unique API key
# gmaps.configure(api_key=gkey)

# url for API

# url = 'https://api.twitter.com/1.1/search/tweets.json'

# # Store a the twitter handle mentions we will use

# handle = '@realDonaldTrump'

# Filter the request for  "geo_enabled": true, language to be only english (lang=en), restrict by region = United Sates
# Reference the url to request the api data, but not going over the 
# request limit per second (max_number_of_requests = 180 per 15 min)
# https://towardsdatascience.com/creating-the-twitter-sentiment-analysis-program-in-python-with-naive-bayes-classification-672e5589a7ed

# convert to JSON object 

# Create a dictionary with the data we need:
            
# print the request to get an idea of the set

# Reference the url for the sentiment analysis code 'https://www.geeksforgeeks.org/twitter-sentiment-analysis-using-python/''



In [3]:
# Authenticate and Make the first API request

client_key = api_key
client_secret = api_secret_key

key_secret = '{}:{}'.format(client_key, client_secret).encode('ascii')
b64_encoded_key = base64.b64encode(key_secret)
b64_encoded_key = b64_encoded_key.decode('ascii')

base_url = 'https://api.twitter.com/'
auth_url = '{}oauth2/token'.format(base_url)

auth_headers = {
    'Authorization': 'Basic {}'.format(b64_encoded_key),
    'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'
}

auth_data = {
    'grant_type': 'client_credentials'
}

auth_resp = requests.post(auth_url, headers=auth_headers, data=auth_data)

auth_resp.json().keys()

access_token = auth_resp.json()['access_token']

search_headers = {
    'Authorization': 'Bearer {}'.format(access_token)    
}


In [4]:
# Adjust parameters for test run
from collections import defaultdict 
search_params = {
    'q': '@realDonaldTrump',
    'lang': 'en',
    'count': 450
}


search_url = '{}1.1/search/tweets.json'.format(base_url)


search_resp = requests.get(search_url, headers=search_headers, params=search_params)

tweet_data = search_resp.json()

tweet_data_filter = defaultdict(list)

statuses = tweet_data['statuses']
processed_tweet_ids = set()


In [5]:
for status in statuses:
    tweet_id = status['id_str']
    if tweet_id in processed_tweet_ids:
        continue
    
    processed_tweet_ids.add(tweet_id)

    tweet_data_filter["UserName"].append(status['user']['screen_name'])
    tweet_data_filter["text"].append(status['text'])
    tweet_data_filter["created_at"].append(status['created_at'])
    tweet_data_filter["followers_count"].append(status['user']['followers_count'])
    tweet_data_filter["retweet_count"].append(status['retweet_count'])
    tweet_data_filter["favorite_count"].append(status['favorite_count'])
    tweet_data_filter["TweetID"].append(tweet_id)
    tweet_data_filter["Retweeted"].append(status['retweeted'])
    tweet_data_filter["UserID"].append(status['user']['id_str'])
    tweet_data_filter["language"].append(status['user']['lang'])
    tweet_data_filter['verified'].append(status['user']['verified'])

tweet_data_filter_pdf = pd.DataFrame(tweet_data_filter)
tweet_data_filter_pdf

,UserName,text,created_at,followers_count,retweet_count,favorite_count,TweetID,Retweeted,UserID,language,verified
0,LargeUglyGuy,"@realDonaldTrump At no point in your rambling,...",Sat Jun 29 16:29:43 +0000 2019,9,0,0,1145006443042693126,False,1136982984639373312,en,False
1,leftareloons,@DavidJHarrisJr @realDonaldTrump @FogCityMidge...,Sat Jun 29 16:29:43 +0000 2019,3228,0,0,1145006441763393538,False,857406786130915330,en,False
2,SonnyRaymond2,RT @realDonaldTrump: 54% in Poll! I would be a...,Sat Jun 29 16:29:43 +0000 2019,10,23328,0,1145006441406713856,False,1083155562924105728,en,False
3,DXLange1,RT @AndyOstroy: @realDonaldTrump You have the ...,Sat Jun 29 16:29:43 +0000 2019,623,311,0,1145006440928534534,False,1105973122686042113,en,False
4,MorganM19609457,RT @PressSec: Today I’ll walk out the gates of...,Sat Jun 29 16:29:43 +0000 2019,9,19953,0,1145006439607525376,False,1125212448934506498,en,False
5,dmadi423,RT @Adam_K_Levin: ⁦.⁦@washingtonpost⁩ ⁦@realDo...,Sat Jun 29 16:29:42 +0000 2019,744,101,0,1145006438227595266,False,275634878,en,False
6,RarnToGoHome,RT @HeARTofGod99: @Helicallight21 @samdernbach...,Sat Jun 29 16:29:42 +0000 2019,25,3,0,1145006436889632769,False,409681619,en,False
7,VernonS43972827,@wvjoe911 @realDonaldTrump Maby Kim can arrang...,Sat Jun 29 16:29:42 +0000 2019,13,0,0,1145006435459198976,False,1084510356729262081,en,False
8,TerryLynnWhite2,RT @DonaldJTrumpJr: The lengths they go to pro...,Sat Jun 29 16:29:42 +0000 2019,22138,1603,0,1145006435455180800,False,4645598232,en,False
9,Smuhamm,RT @realDonaldTrump: Stock Market is heading f...,Sat Jun 29 16:29:41 +0000 2019,133,20032,0,1145006434981167104,False,19142576,en,False


In [11]:
#sentiment analysis for tweet

tweet_data_filter_pdf["TweetTone"] = ""
for index, row in tweet_data_filter_pdf.iterrows():
    tweettext = row['text']
    sent_analysis = TextBlob(tweettext)
       # set sentiment
    if sent_analysis.sentiment.polarity > 0:
        tweet_data_filter_pdf.loc[index, "TweetTone"] = "Positive"
    elif sent_analysis.sentiment.polarity == 0:
        tweet_data_filter_pdf.loc[index, "TweetTone"] = "Neutral"
    else:
        tweet_data_filter_pdf.loc[index, "TweetTone"] = "Negative"

tweet_data_filter_pdf

In [ ]:
tweet_data_filter_pdf.to_excel("TweetFile2.xlsx", engine='xlsxwriter', encoding='utf8')

In [12]:
# Adjust parameters for test run
from collections import defaultdict 

trump_search_params = {
    'screen_name': 'realDonaldTrump',
    'count': 1000
}


user_url = '{}1.1/statuses/user_timeline.json'.format(base_url)


search_resp2 = requests.get(user_url, headers=search_headers, params=trump_search_params)

trump_tweet_data = search_resp2.json()

trump_tweet_data_filter = defaultdict(list)

processed_trump_tweet_ids = set()

# trump_tweet_data

In [13]:
for tweet in trump_tweet_data:
    trump_tweet_id = tweet['id_str']
    if trump_tweet_id in processed_tweet_ids:
        continue
        
    processed_trump_tweet_ids.add(trump_tweet_id)
    
    trump_tweet_data_filter["UserName"].append(tweet['user']['screen_name'])
    trump_tweet_data_filter["text"].append(tweet['text'])
    trump_tweet_data_filter["created_at"].append(tweet['created_at'])
    trump_tweet_data_filter["followers_count"].append(tweet['user']['followers_count'])
    trump_tweet_data_filter["retweet_count"].append(tweet['retweet_count'])
    trump_tweet_data_filter["favorite_count"].append(tweet['favorite_count'])
    trump_tweet_data_filter["TweetID"].append(tweet['id_str'])
    trump_tweet_data_filter["Retweeted"].append(tweet['retweeted'])
    trump_tweet_data_filter["UserID"].append(tweet['user']['id_str'])
    trump_tweet_data_filter["language"].append(tweet['user']['lang'])
    trump_tweet_data_filter['verified'].append(tweet['user']['verified'])


trump_data_filter_pdf = pd.DataFrame(trump_tweet_data_filter)

trump_data_filter_pdf

,UserName,text,created_at,followers_count,retweet_count,favorite_count,TweetID,Retweeted,UserID,language,verified
0,realDonaldTrump,Thank you #G20OsakaSummit https://t.co/9FCqSuR5Bp,Sat Jun 29 10:11:32 +0000 2019,61506048,15666,69621,1144911267641135109,False,25073877,None,True
1,realDonaldTrump,"After some very important meetings, including ...",Fri Jun 28 22:51:41 +0000 2019,61506048,31904,132388,1144740178948493314,False,25073877,None,True
2,realDonaldTrump,54% in Poll! I would be at 75% (with our great...,Fri Jun 28 22:41:17 +0000 2019,61506048,23375,114301,1144737559786020864,False,25073877,None,True
3,realDonaldTrump,"I am in Japan at the G-20, representing our Co...",Fri Jun 28 09:26:34 +0000 2019,61506048,29067,150956,1144537564944228352,False,25073877,None,True
4,realDonaldTrump,The Stock Market went up massively from the da...,Fri Jun 28 09:12:18 +0000 2019,61506048,20943,109499,1144533973428842496,False,25073877,None,True
5,realDonaldTrump,All Democrats just raised their hands for givi...,Fri Jun 28 01:37:04 +0000 2019,61506048,69453,278365,1144419410729242625,False,25073877,None,True
6,realDonaldTrump,Great to be back in Japan for the #G20OsakaSum...,Fri Jun 28 00:46:37 +0000 2019,61506048,15171,76297,1144406713165963264,False,25073877,None,True
7,realDonaldTrump,Bipartisan Humanitarian Aid Bill for the South...,Thu Jun 27 22:22:39 +0000 2019,61506048,24135,103525,1144370485783867392,False,25073877,None,True
8,realDonaldTrump,.....United States Supreme Court is given addi...,Thu Jun 27 17:37:32 +0000 2019,61506048,24893,112799,1144298734311878657,False,25073877,None,True
9,realDonaldTrump,"Seems totally ridiculous that our government, ...",Thu Jun 27 17:37:32 +0000 2019,61506048,28699,130084,1144298731887628288,False,25073877,None,True


In [21]:
trump_data_filter_pdf.to_excel("TrumpFile3.xlsx", engine='xlsxwriter', encoding='utf8')

In [29]:
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [69]:
def remove_punctuation(input_string):
    out_string = ''
    for char in input_string:
        if char not in string.punctuation and char != '\u2026':
#             out_string = out_string + char
#         if char !='...':
            out_string = out_string + char
    return out_string

def removearticles(text):
    re.sub('(\s+)(a|an|and|the)(\s+)', '\1\3', text)

In [70]:
assert callable(remove_punctuation)
assert isinstance(remove_punctuation('a'), str)
assert remove_punctuation("Hey! It's Tom...") == "Hey Its Tom"

In [71]:
# frequently used words
key_words=[]
for text in trump_data_filter_pdf['text']:
    key_words.append(text.split())

keylist = []
for sublist in key_words:
    for item in sublist:
        keylist.append(item)

keylistclean = []

for item in keylist:
    cleanitem = remove_punctuation(item).lower()
    cleanitem2 = removearticles(cleanitem)
    keylistclean.append(cleanitem2)
    
c = dict((word, keylistclean.count(word)) for word in keylistclean)

key_word_trend=pd.DataFrame(c.items(), columns=['key_word', 'occurance'])

key_word_trend.sort_values(by=['occurance'], ascending=False)    

,key_word,occurance
0,None,3556


'httpstco…'

In [27]:
# c = Counter(keylist)

c = dict((word, keylist.count(word)) for word in keylist)

key_word_trend=pd.DataFrame(c.items(), columns=['key_word', 'occurance'])

key_word_trend.sort_values(by=['occurance'], ascending=False)

,key_word,occurance
36,the,141
77,to,91
44,and,73
26,in,57
15,of,53
57,a,50
440,RT,48
22,for,45
123,is,41
17,I,29


In [67]:
trump_data_filter_pdf["TweetTone"] = ""
for index, row in trump_data_filter_pdf.iterrows():
    trumptweettext = row['text']
    sent_analysis = TextBlob(trumptweettext)
       # set sentiment
    if sent_analysis.sentiment.polarity > 0:
        trump_data_filter_pdf.loc[index, "TweetTone"] = "Positive"
    elif sent_analysis.sentiment.polarity == 0:
        trump_data_filter_pdf.loc[index, "TweetTone"] = "Neutral"
    else:
        trump_data_filter_pdf.loc[index, "TweetTone"] = "Negative"
trump_data_filter_pdf        

,UserName,text,created_at,followers_count,retweet_count,favorite_count,TweetID,Retweeted,UserID,language,verified,TweetTone
0,realDonaldTrump,Thank you #G20OsakaSummit https://t.co/9FCqSuR5Bp,Sat Jun 29 10:11:32 +0000 2019,61506048,15666,69621,1144911267641135109,False,25073877,None,True,Neutral
1,realDonaldTrump,"After some very important meetings, including ...",Fri Jun 28 22:51:41 +0000 2019,61506048,31904,132388,1144740178948493314,False,25073877,None,True,Positive
2,realDonaldTrump,54% in Poll! I would be at 75% (with our great...,Fri Jun 28 22:41:17 +0000 2019,61506048,23375,114301,1144737559786020864,False,25073877,None,True,Positive
3,realDonaldTrump,"I am in Japan at the G-20, representing our Co...",Fri Jun 28 09:26:34 +0000 2019,61506048,29067,150956,1144537564944228352,False,25073877,None,True,Negative
4,realDonaldTrump,The Stock Market went up massively from the da...,Fri Jun 28 09:12:18 +0000 2019,61506048,20943,109499,1144533973428842496,False,25073877,None,True,Neutral
5,realDonaldTrump,All Democrats just raised their hands for givi...,Fri Jun 28 01:37:04 +0000 2019,61506048,69453,278365,1144419410729242625,False,25073877,None,True,Negative
6,realDonaldTrump,Great to be back in Japan for the #G20OsakaSum...,Fri Jun 28 00:46:37 +0000 2019,61506048,15171,76297,1144406713165963264,False,25073877,None,True,Positive
7,realDonaldTrump,Bipartisan Humanitarian Aid Bill for the South...,Thu Jun 27 22:22:39 +0000 2019,61506048,24135,103525,1144370485783867392,False,25073877,None,True,Positive
8,realDonaldTrump,.....United States Supreme Court is given addi...,Thu Jun 27 17:37:32 +0000 2019,61506048,24893,112799,1144298734311878657,False,25073877,None,True,Neutral
9,realDonaldTrump,"Seems totally ridiculous that our government, ...",Thu Jun 27 17:37:32 +0000 2019,61506048,28699,130084,1144298731887628288,False,25073877,None,True,Negative


In [66]:
# DATA CLEANING

In [ ]:
# VISUALIZATION REQUEIREMENTS - 6-8

# What is the distribution of sentiment (positive/negative/neutral) per mention of handle?
# Piechart/bar chart per handle mention

# How many (positive/negative/neutral) retweets?


# Scatter plot - size of bubles is number of followers

# Most commom used words by Trump - top keywords
# Use of exclamation points
# Use of Caps lock?
# Emoji symbols he uses

# Sentiment analysis of Trump Tweets
# Sentiment analysis of his tweets for top 5 keyqords he uses (sort and select top 5 based on total count)


# Scatter plot of Trumps tweets colored by sentiment

# Save png versions of our visuals


In [ ]:
# OBSERVATIONS/TRENDS

# Implications - can we predict if he will be elected or not